<a href="https://colab.research.google.com/github/maddy147/reddit_aita/blob/main/senti_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
# from scikitplot.metrics import plot_confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install git+https://github.com/openai/CLIP.git
import clip
nltk.download('stopwords')
nltk.download('wordnet')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-5okg7smw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-5okg7smw
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# posts = pd.read_csv('aita\posts_1k.csv')
# comments = pd.read_csv('aita\comments_1k.csv')
# data = pd.read_excel('aita/data_1k.xlsx',index_col=0)
data = pd.read_excel('/content/drive/MyDrive/nlp/aita/data_1k.xlsx',index_col=0)


In [ ]:
# valid_posts = posts.copy()
# valid_comments = comments.copy()
valid_data = data.copy()
valid_data

,title,body,yta,nta
0,AITA for telling my wife the lock on my daught...,My brother in-law (Sammy) lost his home shortl...,0,123
1,META: This sub is moving towards a value syste...,I’ve enjoyed reading and posting on this sub f...,17,62
2,"UPDATE, AITA for despising my mentally handica...","I'm back like I said I would be,. My [original...",0,44
3,AITA For suing my girlfriend after she had my ...,I'll try to keep this short. I had a [1967 Imp...,0,156
4,UPDATE: AITA for wanting to go to the funeral ...,I want to sincerely thank everyone who comment...,0,0
...,...,...,...,...
993,AITA for leaving in the middle of celebration ...,\n\nI f25 married my husband m31 3 years ago. ...,0,148
994,Aita: For yelling at my fiance for giving my d...,"My best friend Kelly passed away 2 years ago, ...",1,204
995,AITA for labeling all the bottles in the house...,I have two of my children living with me. One ...,1,157
996,AITA for treating my two daughters' pregnancie...,"When my oldest told me she was pregnant, she w...",46,67


In [ ]:
valid_data = valid_data[~(valid_data['title'].str.contains('UPDATE'))]
valid_data = valid_data[~(valid_data['title'].str.contains('Update'))]
valid_data = valid_data[~(valid_data['title'].str.contains('META'))]
valid_data = valid_data[~(valid_data['title'].str.contains('meta'))]
valid_data = valid_data.reset_index()

In [ ]:
valid_data

,index,title,body,yta,nta
0,0,AITA for telling my wife the lock on my daught...,My brother in-law (Sammy) lost his home shortl...,0,123
1,3,AITA For suing my girlfriend after she had my ...,I'll try to keep this short. I had a [1967 Imp...,0,156
2,5,AITA for pretending to get fired when customer...,I am a high schooler with a weekend job at a c...,46,44
3,6,"AITA for bringing up my brother's ""premature"" ...",I am a nurse practitioner and I am the primary...,0,87
4,8,AITA for making a dad joke?,"Note. My step-daughter, Madeline, was about a ...",80,38
...,...,...,...,...,...
884,993,AITA for leaving in the middle of celebration ...,\n\nI f25 married my husband m31 3 years ago. ...,0,148
885,994,Aita: For yelling at my fiance for giving my d...,"My best friend Kelly passed away 2 years ago, ...",1,204
886,995,AITA for labeling all the bottles in the house...,I have two of my children living with me. One ...,1,157
887,996,AITA for treating my two daughters' pregnancie...,"When my oldest told me she was pregnant, she w...",46,67


In [ ]:
#object of WordNetLemmatizer
lm = WordNetLemmatizer()

In [ ]:
def text_transformation(df_col):
    corpus = []
    for item in df_col:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        new_item = new_item.split()
        new_item = [lm.lemmatize(word) for word in new_item if word not in set(stopwords.words('english'))]
        corpus.append(' '.join(str(x) for x in new_item))
    return corpus

In [ ]:
corpus_b = text_transformation(valid_data['body'])
corpus_t = text_transformation(valid_data['title'])

In [ ]:
lbs = [int(y>n) for y,n in zip(valid_data['yta'],valid_data['nta'])]

In [ ]:
class dset(Dataset):
    def __init__(self,txt,lbs) -> None:
        super().__init__()
        self.txt = txt
        self.lbs = lbs
    def __len__(self):
        return len(self.lbs)
    def __getitem__(self, index):
        return self.txt[index],self.lbs[index]

In [ ]:
dataset_t = dset(corpus_t,lbs)
dataset_b = dset(corpus_b,lbs)

In [ ]:
traindata_t,testdata_t = train_test_split(dataset_t,random_state = 42)
traindata_b,testdata_b = train_test_split(dataset_b,random_state = 42)

In [ ]:
print(len(testdata_b))
print(len(testdata_t))

223
223


In [ ]:
model_t, preprocess = clip.load("ViT-B/32", device='cuda')
cls_t = nn.Linear(512,2).to('cuda')

model_b, preprocess = clip.load("ViT-B/32", device='cuda')
cls_b = nn.Linear(512,2).to('cuda')

In [ ]:
train_ldr_t = DataLoader(traindata_t,batch_size = 32)
train_ldr_b = DataLoader(traindata_b,batch_size = 32)

In [ ]:
def run_epoch(train_ldr_t,train_ldr_b,model_t,cls_t,model_b,cls_b,opt):
    ep_loss = 0
    for data_t,data_b in zip(train_ldr_t,train_ldr_b):
        txt_features_t = torch.stack([model_t.encode_text(clip.tokenize(txt,truncate = True).to(device)) for txt in data_t[0]]).squeeze(1).float()
        txt_features_b = torch.stack([model_b.encode_text(clip.tokenize(txt,truncate = True).to(device)) for txt in data_b[0]]).squeeze(1).float()
        loss = F.cross_entropy((cls_t(txt_features_t) + cls_b(txt_features_b))/2,data_t[1].to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        ep_loss += loss.item()

    ep_loss = ep_loss/len(train_ldr_t)

    return ep_loss


def train(train_ldr_t,train_ldr_b,model_t,cls_t,model_b,cls_b,max_epochs):
    losses = []
    opt = torch.optim.SGD(list(model_t.parameters()) + list(cls_t.parameters()) + list(cls_b.parameters()) + list(model_b.parameters()),lr=1e-4)
    for epoch in range(max_epochs):
        ep_loss =run_epoch(train_ldr_t,train_ldr_b,model_t,cls_t,model_b,cls_b,opt)
        print("Loss at epoch {}:{}".format(epoch,ep_loss))
        losses.append(ep_loss)
    return losses

In [ ]:
losses = train(train_ldr_t,train_ldr_b,model_t,cls_t,model_b,cls_b,100)
print(losses)

Loss at epoch 0:0.5411392634823209
Loss at epoch 1:0.4364549971762158
Loss at epoch 2:0.379832162743523
Loss at epoch 3:0.34114626830532435
Loss at epoch 4:0.3130283561490831
Loss at epoch 5:0.2955810342516218
Loss at epoch 6:0.28515298380738213
Loss at epoch 7:0.2775017661707742
Loss at epoch 8:0.27191996858233497
Loss at epoch 9:0.2678939054409663
Loss at epoch 10:0.26478100568056107
Loss at epoch 11:0.26228091510988416
Loss at epoch 12:0.2602504278932299
Loss at epoch 13:0.2585640127460162
Loss at epoch 14:0.25719101833445684
Loss at epoch 15:0.25615447014570236
Loss at epoch 16:0.25528817375500995
Loss at epoch 17:0.2545950763991901
Loss at epoch 18:0.25400748245772864
Loss at epoch 19:0.25350165367126465
Loss at epoch 20:0.2530593840139253
Loss at epoch 21:0.25267538002559115
Loss at epoch 22:0.2523450507294564
Loss at epoch 23:0.2520408108830452
Loss at epoch 24:0.25176047001566204
Loss at epoch 25:0.2515046280764398
Loss at epoch 26:0.2512642790873845
Loss at epoch 27:0.25104563

In [ ]:
torch.save(model_t,'/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/mod_t_100.pt')
torch.save(cls_t,'/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/cls_t_100.pt')
torch.save(model_b,'/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/mod_b_100.pt')
torch.save(cls_b,'/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/cls_b_100.pt')
torch.save(losses,'/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/losses_100.pt')

In [ ]:
model_b = torch.load('/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/mod_b_100.pt',map_location = device)
cls_b = torch.load('/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/cls_b_100.pt',map_location = device)

model_t = torch.load('/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/mod_t_100.pt',map_location = device)
cls_t = torch.load('/content/drive/MyDrive/nlp/models/title_body_as_text/fix_test_data/dif_mod/cls_t_100.pt',map_location = device)

In [ ]:
test_ldr_t =  DataLoader(testdata_t,batch_size= 32)
test_ldr_b =  DataLoader(testdata_b,batch_size= 32)
acc = 0
for d_t,d_b in zip(test_ldr_t,test_ldr_b):
    logits_t = cls_t(torch.stack([model_t.encode_text(clip.tokenize(txt,truncate = True).to(device)) for txt in d_t[0]]).squeeze(1).float())
    logits_b = cls_b(torch.stack([model_b.encode_text(clip.tokenize(txt,truncate = True).to(device)) for txt in d_b[0]]).squeeze(1).float())
    pred = torch.argmax((logits_t+logits_b)/2,dim = 1)
    acc = acc + sum(pred == d_t[1].to(device))/(32)

In [ ]:
acc/len(test_ldr_t)

tensor(0.9152, device='cuda:0')